# Issue2
Die Funktion soll folgendes tun:
Diese Funktion liest einen gesamten Modultext ein und macht daraus einen Dictionary. Möglicherweise ist dafür regex am
Sinnvollsten.
Diese Funktion kann dann innerhalb der Funktion aus issue#1 verwendet werden, und den dict erstellen, der dann im .json 
file gespeichert wird.

Fehler: 
    IndexError: list index out of range
    
        Ich gehe davon aus, dass das Programm ein Problem hat, weil in der Spalte nichts drinnen steht


In [3]:
import json
import re

def split_text_file(filename):
    encodings = ['utf-8', 'latin-1', 'iso-8859-1', 'cp1252']  # List of common codecs

    content = None
    for encoding in encodings:
        try:
            with open(filename, 'r', encoding=encoding) as file:
                content = file.read()
            break  # Break if reading was successful
        except UnicodeDecodeError:
            continue  # Try the next encoding
        except FileNotFoundError:
            raise FileNotFoundError(f"The file {filename} was not found.")
    
    if content is None:
        raise ValueError("Failed to read the file with the given encodings.")

    # Ensure the delimiter 'B' is included at the beginning of each section
    sections = content.split('\n\n\n\nB')
    
    # Add the 'B' back to the start of each section except the first one
    sections = [sections[0]] + ['B' + sec for sec in sections[1:]]

    # Remove leading and trailing whitespace from each section
    sections = [sec.strip() for sec in sections]

    return sections

def generate_filename_from_first_line(section):
    first_line = section.split('\n', 1)[0].strip()
    # Remove invalid filename characters
    filename = "".join(x for x in first_line if (x.isalnum() or x in "._- ")).strip()[:50]
    return filename

def parse_section_to_dict(section):
    section_dict = {}
    
    # Extract Titel
    titel_match = re.search(r'^(BP\d+\n.*?)(?=\n\n|$)', section, re.DOTALL)
    if titel_match:
        section_dict["Titel"] = [line.strip() for line in titel_match.group(1).split('\n')]

    # Extract Inhalte
    inhalte_match = re.search(r'Inhalte\n\n(.*?)(?=\n\n[A-Z])', section, re.DOTALL)
    if inhalte_match:
        section_dict["Inhalte"] = inhalte_match.group(1).replace('\n', ' ').strip()

    # Extract Lernergebnisse / Kompetenzziele
    lernergebnisse_match = re.search(r'Lernergebnisse / Kompetenzziele\n\n(.*?)(?=\n\n[A-Z])', section, re.DOTALL)
    if lernergebnisse_match:
        section_dict["Lernergebnisse / Kompetenzziele"] = lernergebnisse_match.group(1).replace('\n', ' ').strip()

    # Extract Teilnahmevoraussetzungen
    teilnahmevoraussetzungen_match = re.search(r'Teilnahmevoraussetzungen für Modul bzw. für einzelne Lehrveranstaltungen des Moduls\n\n(.*?)(?=\n\n[A-Z])', section, re.DOTALL)
    if teilnahmevoraussetzungen_match:
        section_dict["Teilnahmevoraussetzungen für Modul bzw. für einzelne Lehrveranstaltungen des Moduls"] = teilnahmevoraussetzungen_match.group(1).replace('\n', ' ').strip()

    # Extract Empfohlene Voraussetzungen
    empfohlene_voraussetzungen_match = re.search(r'Empfohlene Voraussetzungen\n\n(.*?)(?=\n\n[A-Z])', section, re.DOTALL)
    if empfohlene_voraussetzungen_match:
        section_dict["Empfohlene Voraussetzungen"] = empfohlene_voraussetzungen_match.group(1).replace('\n', ' ').strip()

    # Extract Organisatorische Hinweise
    organisatorische_hinweise_match = re.search(r'Organisatorische Hinweise\n\n(.*?)(?=\n\n[A-Z])', section, re.DOTALL)
    if organisatorische_hinweise_match:
        section_dict["Organisatorische Hinweise"] = organisatorische_hinweise_match.group(1).replace('\n', ' ').strip()

    # Extract Zuordnung des Moduls
    zuordnung_moduls_match = re.search(r'Zuordnung des Moduls \(Studiengang / Fachbereich\)\n(.*?)(?=\n\n[A-Z])', section, re.DOTALL)
    if zuordnung_moduls_match:
        section_dict["Zuordnung des Moduls (Studiengang / Fachbereich)"] = zuordnung_moduls_match.group(1).replace('\n', ' ').strip()

    # Extract Verwendbarkeit des Moduls
    verwendbarkeit_moduls_match = re.search(r'Verwendbarkeit des Moduls \n(.*?)(?=\n\n[A-Z])', section, re.DOTALL)
    if verwendbarkeit_moduls_match:
        section_dict["Verwendbarkeit des Moduls für andere Studiengänge"] = verwendbarkeit_moduls_match.group(1).replace('\n', ' ').strip()

    # Extract Häufigkeit des Angebots
    haeufigkeit_angebots_match = re.search(r'Häufigkeit des Angebots\n(.*?)(?=\n\n[A-Z])', section, re.DOTALL)
    if haeufigkeit_angebots_match:
        section_dict["Häufigkeit des Angebots"] = haeufigkeit_angebots_match.group(1).replace('\n', ' ').strip()

    # Extract Dauer des Moduls
    dauer_moduls_match = re.search(r'Dauer des Moduls\n(.*?)(?=\n\n[A-Z])', section, re.DOTALL)
    if dauer_moduls_match:
        section_dict["Dauer des Moduls"] = dauer_moduls_match.group(1).replace('\n', ' ').strip()

    # Extract Modulbeauftragte / Modulbeauftragter
    modulbeauftragte_match = re.search(r'Modulbeauftragte / Modulbeauftragter\n(.*?)(?=\n\n[A-Z])', section, re.DOTALL)
    if modulbeauftragte_match:
        section_dict["Modulbeauftragte / Modulbeauftragter"] = modulbeauftragte_match.group(1).replace('\n', ' ').strip()

    # Extract Studiennachweise/ ggf. als Prüfungsvorleistungen
    studiennachweise_match = re.search(r'Studiennachweise/ ggf. als Prüfungsvorleistungen\n\n(.*?)(?=\n\n[A-Z])', section, re.DOTALL)
    if studiennachweise_match:
        studiennachweise_text = studiennachweise_match.group(1).replace('\n', ' ').strip()
        studiennachweise_parts = studiennachweise_text.split('Leistungsnachweise')
        if len(studiennachweise_parts) == 2:
            section_dict["Studiennachweise/ ggf. als Prüfungsvorleistungen"] = {
                "Teilnahmenachweise": studiennachweise_parts[0].strip() if studiennachweise_parts[0].strip() else "keine",
                "Leistungsnachweise": studiennachweise_parts[1].strip()
            }

    # Extract Lehr- / Lernformen
    lehr_lernformen_match = re.search(r'Lehr- / Lernformen\n(.*?)(?=\n\n[A-Z])', section, re.DOTALL)
    if lehr_lernformen_match:
        section_dict["Lehr- / Lernformen"] = lehr_lernformen_match.group(1).replace('\n', ' ').strip()

    # Extract Unterrichts- / Prüfungssprache
    unterricht_pruefungssprache_match = re.search(r'Unterrichts- / Prüfungssprache\n(.*?)(?=\n\n[A-Z])', section, re.DOTALL)
    if unterricht_pruefungssprache_match:
        section_dict["Unterrichts- / Prüfungssprache"] = unterricht_pruefungssprache_match.group(1).replace('\n', ' ').strip()

    # Extract Modulprüfung
    modulpruefung_match = re.search(r'Modulprüfung\n\n(.*?)(?=\n\n[A-Z])', section, re.DOTALL)
    if modulpruefung_match:
        modulpruefung_text = modulpruefung_match.group(1).replace('\n', ' ').strip()
        modulpruefung_parts = modulpruefung_text.split('kumulative Modulprüfung bestehend aus:')
        if len(modulpruefung_parts) == 2:
            section_dict["Modulprüfung"] = {
                "Modulabschlussprüfung bestehend aus:": modulpruefung_parts[0].strip(),
                "kumulative Modulprüfung bestehend aus:": modulpruefung_parts[1].strip()
            }

    # Extract Übersicht
    ueberblick_match = re.search(r'LV-Form\nSWS\nCP\nSemester\n(.*?)(?=\n\n[A-Z])', section, re.DOTALL)
    if ueberblick_match:
        ueberblick_text = ueberblick_match.group(1).replace('\n', ' ').strip()
        ueberblick_lines = ueberblick_text.split('\n')
        ueberblick_dict = {
            "Veranstaltung": ueberblick_lines[0].split(),
            "LV-Form": ueberblick_lines[1].split(),
            "SWS": ueberblick_lines[2].split(),
            "CP": ueberblick_lines[3].split(),
            "1": ueberblick_lines[4].split(),
            "2": ueberblick_lines[5].split(),
            "3": ueberblick_lines[6].split(),
            "4": ueberblick_lines[7].split(),
            "5": ueberblick_lines[8].split(),
            "6": ueberblick_lines[9].split(),
        }
        section_dict["Übersicht"] = ueberblick_dict

    return section_dict

def process_file(filename):
    sections = split_text_file(filename)

    all_sections = []
    for section in sections:
        section_dict = parse_section_to_dict(section)
        all_sections.append(section_dict)
    
    with open('output.json', 'w', encoding='utf-8') as f:
        json.dump(all_sections, f, ensure_ascii=False, indent=4)

# Beispiel zur Verarbeitung einer Datei
process_file('modulhandbuch.kurz.txt')


IndexError: list index out of range